# Set Up (imports including psspy)

In [2]:
# uncomment to install for first time
# !pip install pandas
# !pip install numpy

In [3]:
import pandas as pd
import numpy as np
import os, sys
print(sys.version) # should be 3.11.9

3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


In [4]:
print(sys.path)

['C:\\Users\\lukas\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\lukas\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\lukas\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\lukas\\AppData\\Local\\Programs\\Python\\Python311', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv', '', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv\\Lib\\site-packages', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\lukas\\Documents\\Personal Projects\\.venv\\Lib\\site-packages\\Pythonwin']


In [5]:
PSSE_LOCATION = r"C:\Program Files\PTI\PSSE36\36.1\PSSPY311" # this is where the psspy module is located on my machine
sys.path.append(PSSE_LOCATION)
os.environ['PATH'] += ';' + PSSE_LOCATION

In [6]:
import psse3601
import psspy

# Import Case Data

In [7]:
# initialize system
psspy.psseinit(39)


 PSS(R)E Xplore Version 36
 Copyright (c) 1976-2025
 Siemens Industry, Inc.,
 Power Technologies International                            (PTI)
 This program is a confidential  unpublished  work  created  and  first
 licensed in 1976.  It is a trade secret which is the property of  PTI.
 All use,  disclosure,  and/or reproduction not specifically authorized
 by  PTI  is prohibited.   This  program is protected  under  copyright
 laws  of  non-U.S.  countries  and  by  application  of  international
 treaties.  All  Rights  Reserved  Under  The  Copyright  Laws.


           SIEMENS POWER TECHNOLOGIES INTERNATIONAL

    50 BUS POWER SYSTEM SIMULATOR--PSS(R)E Xplore-36.1.0

             INITIATED ON SAT, APR 26 2025  22:15


0

In [8]:
# read case
psspy.read(0, "IEEE 39 bus - More Variables.RAW")


 Reading IC, SBASE, REV, XFRRAT, NXFRAT, BASFRQ...

 Reading two line case heading...

 Reading system-wide data...

 Reading bus data...

 Reading load data...

 Reading fixed bus shunt data...

 Reading voltage droop control data...

 Reading generator data...

 Reading Switching Device Ratings Table data...

 Reading non-transformer branch data...

 Reading system switching device data...

 Reading transformer data...

 Reading area interchange data...

 Reading two-terminal dc line data...

 Reading VSC dc line data...

 Reading transformer impedance correction data...

 Reading multi-terminal dc line data...

 Reading multi-section line data...

 Reading zone name data...

 Reading inter-area transfer data...

 Reading owner name data...

 Reading FACTS device data...

 Reading switched shunt data...

 Reading GNE device data...

 Reading induction machine data...

 Reading load type data...

 Reading interface data...

 Reading interface element data...

 Reading substation data

0

# Testing

### Run load flow analysis

In [9]:
# run load flow analysis
psspy.fnsl([0,0,0,1,1,0,99,0]) # flat start newton-raphson load flow


 Ordering network...
 Diagonals = 38   Off-diagonals = 66   Maximum size = 90

  ITER       DELTAP      BUS         DELTAQ      BUS        DELTA/V/      BUS       DELTAANG      BUS
   0         0.0004(     16     )    0.0032(      5     )
                                                             0.00000(      8     )   0.00000(     28     )
   1         0.0000(      2     )    0.0000(      5     )
                                                             0.00000(      8     )   0.00000(     39     )
   2         0.0000(      6     )    0.0000(     10     )


 Reached tolerance in 2 iterations

 Largest mismatch:     -0.00 MW      0.00 Mvar      0.00 MVA at bus 5 [5           1.0000]
 System total absolute mismatch:                    0.00 MVA

 SWING BUS SUMMARY:
   BUS#-SCT X-- NAME --X BASKV      PGEN     PMAX    PMIN      QGEN     QMAX    QMIN
     31     31          1.0000     571.3    750.0     0.0     363.9    800.0  -500.0


0

## Retrieve bus and machine data from base system

In [10]:
from itertools import groupby
from operator import itemgetter
attr_type = itemgetter(0)

def subsystem_info(name, attributes, sid=-1, inservice=True):
    """
    Returns requested attributes from the PSS(r)E subsystem API
    for the given subsystem id and subsystem element name.
    e.g. to retrieve bus attributes "NAME", "NUMBER" and "PU"
      
      subsystem_info('bus', ["NAME", "NUMBER", "PU"])
    where the 'bus' `name` argument comes from the original
    PSS(r)E subsystem API naming convention found in Chapter 8 of the
    PSS(r)E API.
    abusint  # bus
    amachint # mach
    aloadint # load
    Args:
      inservice [optional]: True (default) to list only information
         for in service elements;
      sid [optional]: list only information for elements in this
         subsystem id (-1, all elements by default).  
    
    """
    name = name.lower()
    gettypes = getattr(psspy, 'a%stypes' % name)
    apilookup = {
            'I': getattr(psspy, 'a%sint' % name),
            'R': getattr(psspy, 'a%sreal' % name),
            'X': getattr(psspy, 'a%scplx' % name),
            'C': getattr(psspy, 'a%schar' % name), }

    result = []
    ierr, attr_types = gettypes(attributes)

    df = pd.DataFrame()

    for k, group in groupby(zip(attr_types, attributes), key=attr_type):
      func = apilookup[k]
      strings = list(list(zip(*group))[1])
      ierr, res = func(sid, flag=1 if inservice else 2, string=strings)
      if len(strings) == 0:
        df[strings[0]] = res[0]
      else:
         for i in range(0, len(strings)):
            df[strings[i]] = res[i]
    return df


In [11]:
businfo = subsystem_info('bus', ['NUMBER', 'PU', 'Angle'], sid=-1)
businfo.head()

,NUMBER,PU,Angle
0,1,1.051784,-0.149610
1,2,1.059875,-0.106962
2,3,1.056548,-0.156151
3,4,1.059186,-0.170671
4,5,1.073595,-0.153021


In [12]:
geninfo = subsystem_info('mach', ['NUMBER', 'ID', 'PMAX', 'PMIN', 'PGEN', 'QMAX', 'QMIN', 'QGEN'], sid=-1)
geninfo['BusNum-GenID'] = geninfo['NUMBER'].astype(str) + '-' + geninfo['ID'].astype(str)
geninfo.sort_values(by='PGEN', ascending=False).head()

,NUMBER,ID,PMAX,PMIN,PGEN,QMAX,QMIN,QGEN,BusNum-GenID
9,39,1,1500.0,0.0,1000.0,1500.0,-1000.0,-36.490406,39-1
8,38,1,1000.0,0.0,830.0,800.0,-500.0,-0.473640,38-1
2,32,1,750.0,0.0,650.0,800.0,-500.0,1.529850,32-1
5,35,1,750.0,0.0,650.0,800.0,-500.0,167.044128,35-1
3,33,1,750.0,0.0,632.0,800.0,-500.0,69.666168,33-1


In [13]:
loadinfo = subsystem_info('load', ['NUMBER', 'ID', 'MVAACT'], sid=-1)
loadinfo['MW'] = loadinfo['MVAACT'].apply(lambda x: x.real)
loadinfo['MVAR'] = loadinfo['MVAACT'].apply(lambda x: x.imag)
loadinfo['BusNum-LoadID'] = loadinfo['NUMBER'].astype(str) + '-' + loadinfo['ID'].astype(str)
loadinfo.sort_values(by='MVAACT', ascending=False).head()

,NUMBER,ID,MVAACT,MW,MVAR,BusNum-LoadID
30,39,1,1104.000000+ 250.000000j,1104.000000,250.000000,39-1
19,20,1,680.000000+ 103.000000j,680.000000,103.000000,20-1
7,8,1,522.000000+ 176.000000j,522.000000,176.000000,8-1
3,4,1,500.000000+ 184.000000j,500.000000,184.000000,4-1
15,16,1,329.399994+ 32.299999j,329.399994,32.299999,16-1


## Rune line limit report

In [14]:
psspy.report_output(2, r"line_limit_report.txt", [0,0])
psspy.rate_3(0,1,1,1,1,0,100.0,0)
psspy.report_output(1, "", [])


 Output completed


0

In [15]:
def clean_line_limit_report(df):
    # Show the first few rows to confirm
    old_cols = df.columns
    updated_cols = []
    for col in old_cols[0:5]:
        updated_cols.append('from_bus ' + col)
    for col in old_cols[5:10]:
        updated_cols.append('to_bus ' + col)
    updated_cols.extend(old_cols[10:])
    df.columns = updated_cols
    df.drop(['from_bus X--', 'to_bus X--.1', 'from_bus AREA','to_bus AREA.1', 'CKT', 'RATE3','PERCENT.1'], axis=1, inplace=True)
    return df

line_limit_report = pd.read_fwf("line_limit_report.txt", skiprows=10)
line_limit_report = clean_line_limit_report(line_limit_report)
line_limit_report.head()

,from_bus BUS#-SCT,from_bus NAME,from_bus --X BASKV,to_bus BUS#-SCT.1,to_bus NAME.1,to_bus --X BASKV.1,LOADING,RATE1,PERCENT
0,1,NaN,1.0000*,2,NaN,1.0000,125.5,1000.0,12.5
1,1,NaN,1.0000,39,NaN,1.0000*,169.5,1000.0,16.9
2,2,NaN,1.0000*,3,NaN,1.0000,365.0,1000.0,36.5
3,2,NaN,1.0000,25,NaN,1.0000*,265.6,1000.0,26.6
4,3,NaN,1.0000*,4,NaN,1.0000,80.8,1000.0,8.1


## Add a load and a generator to a bus

In [16]:
# save next load id and next generator id for each bus
bus_next_load_and_generator_ids = {}
bus_next_generator_id = {}
buses = businfo['NUMBER'].tolist()

for bus in buses:

    existing_load_id = loadinfo[loadinfo['NUMBER'] == bus]['ID'].max()
    if not pd.isna(existing_load_id):
        next_load_id = str(int(existing_load_id) + 1)
    else:
        next_load_id = str(1)

    existing_generator_id = geninfo[geninfo['NUMBER'] == bus]['ID'].max()
    if not pd.isna(existing_generator_id):
        next_generator_id = str(int(existing_generator_id) + 1)
    else:
        next_generator_id = str(1)
    
    bus_next_load_and_generator_ids[bus] = {'next_load_id': next_load_id, 'next_generator_id': next_generator_id}

In [17]:
bus = 1
id = '2'

p_load = 500
q_load = 0
psspy.load_data_3(bus, '2', [1, 1, 1, 1, 1], [p_load, q_load, 0, 0, 0, 0])

p_gen = 300
q_gen = 0
q_max = 300
q_min = 0
p_max = 400
p_min = 0
psspy.plant_data_4(bus, 0, [0, 0], [1, 100])
psspy.machine_data_3(bus, id, [1, 1, 0, 0, 0, 0, 0], [p_gen, q_gen, q_max, q_min, p_max, p_min])


 Load "2" at bus 1 [1           1.0000] added. Power flow data items with non-default values:
 X--DEFAULT---X  X---ACTUAL---X  DATA ITEM
    0.00000         500.000      PL

 Plant at bus 1 [1           1.0000] added. Data items with non-default values:
 X--DEFAULT---X  X---ACTUAL---X  DATA ITEM
          0               1      IREG

 Machine "2" at bus 1 [1           1.0000] added. Power flow data items with non-default values:
 X--DEFAULT---X  X---ACTUAL---X  DATA ITEM
    0.00000         300.000      PG
    9999.00         300.000      QT
   -9999.00         0.00000      QB
    9999.00         400.000      PT
   -9999.00         0.00000      PB


0

# Perform Looping

In [18]:
# load data params
p_load = 500
q_load = 0

# generator data params
p_gen = 300
q_gen = 0
q_max = 300
q_min = 0
p_max = 400
p_min = 0

In [ ]:
# loop over each bus and run simulation
for load_bus in buses:

    # create directory
    directory = 'bus_{0}_simulations'.format(load_bus)
    if not os.path.isdir(directory):
        os.mkdir(directory)

    ###################################################################################################################################

    # start with original case
    psspy.read(0, "IEEE 39 bus - More Variables.RAW")

    # add load to bus
    psspy.load_data_3(load_bus, bus_next_load_and_generator_ids[load_bus]['next_load_id'], [1, 1, 1, 1, 1], [p_load, q_load, 0, 0, 0, 0])

    # first, run power flow model without a generator
    psspy.fnsl([0,0,0,1,1,0,99,0]) # flat start newton-raphson load flow

    # save bus data
    businfo = subsystem_info('bus', ['NUMBER', 'PU', 'Angle'], sid=-1)
    businfo.to_csv(directory+r'\bus_data_no_generator.csv', index=False)

    # save generator data
    geninfo = subsystem_info('mach', ['NUMBER', 'ID', 'PMAX', 'PMIN', 'PGEN', 'QMAX', 'QMIN', 'QGEN'], sid=-1)
    geninfo['BusNum-GenID'] = geninfo['NUMBER'].astype(str) + '-' + geninfo['ID'].astype(str)
    geninfo.to_csv(directory+r'\gen_data_no_generator.csv', index=False)

    psspy.report_output(2, directory+r"\line_limit_report_no_generator.txt", [0,0])
    psspy.rate_3(0,1,1,1,1,0,100.0,0)
    psspy.report_output(1, "", [])
    line_limit_report = pd.read_fwf(directory+r"\line_limit_report_no_generator.txt", skiprows=10)
    line_limit_report = clean_line_limit_report(line_limit_report)
    line_limit_report.to_csv(directory+r'\line_limit_report_no_generator.csv', index=False)

    ###################################################################################################################################

    # now, loop over each bus and add a generator
    for gen_bus in buses:

        # start with original case
        psspy.read(0, "IEEE 39 bus - More Variables.RAW")

        # add load to bus
        psspy.load_data_3(load_bus, bus_next_load_and_generator_ids[load_bus]['next_load_id'], [1, 1, 1, 1, 1], [p_load, q_load, 0, 0, 0, 0])

        # add generator to bus
        psspy.plant_data_4(gen_bus, 0, [0, 0], [1, 100])
        psspy.machine_data_3(gen_bus, bus_next_load_and_generator_ids[gen_bus]['next_generator_id'], [1, 1, 0, 0, 0, 0, 0], [p_gen, q_gen, q_max, q_min, p_max, p_min])

        # run power flow model with generator
        psspy.fnsl([0,0,0,1,1,0,99,0]) # flat start newton-raphson load flow

        # save bus data
        businfo = subsystem_info('bus', ['NUMBER', 'PU', 'Angle'], sid=-1)
        businfo.to_csv(directory+r'\bus_data_with_generator_on_bus_{0}.csv'.format(gen_bus), index=False)

        # save generator data
        geninfo = subsystem_info('mach', ['NUMBER', 'ID', 'PMAX', 'PMIN', 'PGEN', 'QMAX', 'QMIN', 'QGEN'], sid=-1)
        geninfo['BusNum-GenID'] = geninfo['NUMBER'].astype(str) + '-' + geninfo['ID'].astype(str)
        geninfo.to_csv(directory+r'\gen_data_with_generator_on_bus_{0}.csv'.format(gen_bus), index=False)

        psspy.report_output(2, directory+r"\line_limit_report_with_generator_on_bus_{0}.txt".format(gen_bus), [0,0])
        psspy.rate_3(0,1,1,1,1,0,100.0,0)
        psspy.report_output(1, "", [])
        line_limit_report = pd.read_fwf(directory+r"\line_limit_report_with_generator_on_bus_{0}.txt".format(gen_bus), skiprows=10)
        line_limit_report = clean_line_limit_report(line_limit_report)
        line_limit_report.to_csv(directory+r"\line_limit_report_with_generator_on_bus_{0}.csv".format(gen_bus), index=False)


 Reading IC, SBASE, REV, XFRRAT, NXFRAT, BASFRQ...

 Reading two line case heading...

 Reading system-wide data...

 Reading bus data...

 Reading load data...

 Reading fixed bus shunt data...

 Reading voltage droop control data...

 Reading generator data...

 Reading Switching Device Ratings Table data...

 Reading non-transformer branch data...

 Reading system switching device data...

 Reading transformer data...

 Reading area interchange data...

 Reading two-terminal dc line data...

 Reading VSC dc line data...

 Reading transformer impedance correction data...

 Reading multi-terminal dc line data...

 Reading multi-section line data...

 Reading zone name data...

 Reading inter-area transfer data...

 Reading owner name data...

 Reading FACTS device data...

 Reading switched shunt data...

 Reading GNE device data...

 Reading induction machine data...

 Reading load type data...

 Reading interface data...

 Reading interface element data...

 Reading substation data

# Analyze results

In [ ]:
# loop over each bus and analyze simulation

# goal here is to pick best and worst scenarios in terms of line loading. Maybe also plot all line loading points into histogram, colored by simulation number.
# could also make a couple graphs about generator bus data (voltage pu, angle) and generator data (reactive power generation) and slack bus data

for load_bus in buses:

    directory = 'bus_{0}_simulations'.format(load_bus)